In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from tqdm import tqdm

url_match = input('introduce la url del partido: ')
# por ejemplo: https://es.besoccer.com/partido/brentford/arsenal/202318282

r_match = requests.get(url_match)

s_match = BeautifulSoup(r_match.text, 'lxml')

find_analisis = s_match.find('div', attrs={'class':'menu-scroll'}).select('a[href*="analisis"]')
analisis = find_analisis[0]

link_analisis = analisis.get('href')

r_link_analisis = requests.get(link_analisis)

s_link_analisis = BeautifulSoup(r_link_analisis.text, 'lxml')

def obtener_stats(s_link_analisis):
    stats_dict = {}
    
    date = s_link_analisis.find('div', attrs={'starttime':'info match-link'})
    if date:
        stats_dict['date'] = date.get_text('/', strip=True)
    else:
        stats_dict['date'] = None 
        
    time = s_link_analisis.find('div', attrs={'class':'data'})
    if time:
        stats_dict['time'] = time.get_text('/', strip=True)
    else:
        stats_dict['time'] = None
        
    competition = s_link_analisis.find('h3', attrs={'class':'competition'})
    if competition:
        stats_dict['competition'] = competition.get_text('/', strip=True)
    else:
        stats_dict['competition'] = None
        
    homeTeam = s_link_analisis.find('p', attrs={'itemprop':'homeTeam'})
    if homeTeam:
        stats_dict['homeTeam'] = homeTeam.get_text('/', strip=True)
    else:
        stats_dict['homeTeam'] = None
    
    homeRank = s_link_analisis.find('div', attrs={'class':'position t1'})
    if homeRank:
        stats_dict['homeRank'] = homeRank.get_text()
    else:
        stats_dict['homeRank'] = None    
    
    homeGoals = s_link_analisis.find('div', attrs={'class':'content-box t-1'}).find('div', attrs={'class':'poss-box'})
    if homeGoals:
        stats_dict['homeGoals'] = homeGoals.get_text('/', strip=True)
    else:
        stats_dict['homeGoals'] = None     
    
    awayTeam = s_link_analisis.find('p', attrs={'itemprop':'awayTeam'})
    if awayTeam:
        stats_dict['awayTeam'] = awayTeam.get_text('/', strip=True)
    else:
        stats_dict['awayTeam'] = None     
        
    awayRank = s_link_analisis.find('div', attrs={'class':'position t2'})
    if awayRank:
        stats_dict['awayRank'] = awayRank.get_text()
    else:
        stats_dict['awayRank'] = None     
        
    awayGoals = s_link_analisis.find('div', attrs={'class':'content-box t-2'}).find('div', attrs={'class':'poss-box'})
    if awayGoals:
        stats_dict['awayGoals'] = awayGoals.get_text('/', strip=True)
    else:
        stats_dict['awayGoals'] = None     
 
    draw = s_link_analisis.find('div', attrs={'class':'content-box draw'}).find('div', attrs={'class':'cont'})
    if draw:
        stats_dict['draw'] = draw.get_text('/', strip=True)
    else:
        stats_dict['draw'] = None    
    
    return stats_dict

obtener_stats(s_link_analisis)

df = pd.DataFrame(obtener_stats(s_link_analisis), index=list(range(0,1)))

name_csv = s_link_analisis.find('p', attrs={'itemprop':'homeTeam'}).get_text('/', strip=True)
    
df.to_csv(f'{name_csv}.csv', header=False)

df.head()

introduce la url del partido: https://es.besoccer.com/partido/brentford/arsenal/202318282


date   time                 competition   homeTeam homeRank  \
0  None  13:00  Premier League.  Jornada 8  Brentford       8º   

                                      homeGoals awayTeam awayRank  \
0  14.8%/Probabilidad gana/0.92/Goles esperados  Arsenal       1º   

                                      awayGoals      draw  
0  65.9%/Probabilidad gana/2.17/Goles esperados  0-0/4.6%